# Status

In [1]:
import mysql.connector
import json
import time
def get_connection():
    return mysql.connector.connect(
        host="kadman-database.c8boumioc8hx.us-east-1.rds.amazonaws.com",
        user="admin",
        password="12345678",
        database="kadmandb",
        port=3306,
        charset='utf8mb4'
    )


In [2]:
con = get_connection()
cursor = con.cursor()


In [ ]:
# Query all vendor data
cursor.execute("ALTER TABLE vendors DROP COLUMN lineID")
rows = cursor.fetchall()
print(rows)

In [16]:
cursor.execute("ALTER TABLE vendors ADD COLUMN UserProfile VARCHAR(20)")
con.commit()
cursor.close()
con.close()

In [6]:
cursor = con.cursor()
cursor.execute("SELECT * FROM layouts")

# Get all rows
rows = cursor.fetchall()

# Get column names
column_names = [desc[0] for desc in cursor.description]

# Convert rows to list of dicts
result = [dict(zip(column_names, row)) for row in rows]

json_output = json.dumps(result, ensure_ascii=False, indent=2)
# Optional: Save to file
with open("layouts_dump.json", "w", encoding="utf-8") as f:
    f.write(json_output)

In [54]:
conn = wait_for_db_connection()

try:
    cursor
except NameError:
    cursor = None

if cursor is not None:
    try:
        # fetch all unread results if any
        cursor.fetchall()
    except Exception:
        # ignore if no results or error
        pass
    cursor.close()
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS waiting_vendors")
print("✅ Table 'waiting_vendors' dropped successfully.")
cursor.close()
conn.commit()
cursor.close()


✅ Successfully connected to MySQL database
✅ Table 'waiting_vendors' dropped successfully.


False

In [2]:
def create_waiting_vendors_table():
    conn = get_connection()
    cursor = conn.cursor()

    create_table_sql = """
    CREATE TABLE IF NOT EXISTS waiting_vendors (
        LineID VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        UserProfile VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        PRIMARY KEY (LineID)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """

    try:
        cursor.execute(create_table_sql)
        print("✅ Table 'waiting_vendors' created successfully.")
    except mysql.connector.Error as err:
        print(f"❌ Error creating table: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
create_waiting_vendors_table()


✅ Table 'waiting_vendors' created successfully.


In [22]:
def show_all_rows():
    conn = get_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT * FROM layouts")
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]  # Current column names

        # Desired order
        desired_order = ["id", "name", "status", "data"]

        # Create mapping of column name -> index
        col_index = {col: idx for idx, col in enumerate(columns)}

        if rows:
            print(f"Rows in 'layouts' ({len(rows)} rows):\n")
            print(" | ".join(desired_order))  # Print headers
            print("-" * (len(" | ".join(desired_order)) + 10))

            for row in rows:
                # Reorder values according to desired_order
                reordered = [str(row[col_index[col]]) for col in desired_order]
                print(" | ".join(reordered))
        else:
            print("No rows found in table.")
    except mysql.connector.Error as err:
        print(f"Error fetching rows: {err}")
    finally:
        cursor.close()
        conn.close()


# Run the function
show_all_rows()


Rows in 'layouts' (6 rows):

id | name | status | data
-----------------------------------
1 | layout1 | inactive | {"A1": {"status": "", "vendorID": 1, "shop_name": "อิงสร้อยผ้าโพก"}, "A2": {"status": "", "vendorID": 2, "shop_name": "โอเสื้อยืด,เชิ้ต,แว่นตา"}, "A3": {"status": "", "vendorID": 3, "shop_name": "ไอซ์ stronger.store25 เสื้อยืดแจ็คเก็ต"}, "A4": {"status": "", "vendorID": 4, "shop_name": "พี่ตั้มหอยทากเสื้อยืดของเล่น"}, "A5": {"status": "", "vendorID": 5, "shop_name": "แม่ต่ายเสื้อยืด"}, "A6": {"status": "", "vendorID": 6, "shop_name": "แม่ต่ายแขนยาว"}, "A7": {"status": "", "vendorID": 7, "shop_name": "โอมเพี้ยงเสื้อยืด"}, "A8": {"status": "", "vendorID": 8, "shop_name": "พี่เบนซ์กระเป๋า"}, "A9": {"status": "", "vendorID": 9, "shop_name": "ฟ่าง y2k"}, "B1": {"status": "", "vendorID": 32, "shop_name": "พี่เอ๋เสื้อเชิ้ตฮาวาย"}, "B2": {"status": "", "vendorID": 33, "shop_name": "จ๊อยซ์กระเป๋า"}, "B3": {"status": "", "vendorID": 34, "shop_name": "เจ๊ก้อยของเล่นเด็ก"}, "B4": {"s

In [25]:
def show_all_rows():
    conn = get_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT * FROM vendors")
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]  # Get column names

        if rows:
            print(f"Rows in 'vendors' ({len(rows)} rows):\n")
            print(" | ".join(columns))  # Print column headers
            print("-" * (len(" | ".join(columns)) + 10))
            for row in rows:
                print(" | ".join(str(cell) for cell in row))
        else:
            print("No rows found in table.")
    except mysql.connector.Error as err:
        print(f"Error fetching rows: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
show_all_rows()
    

Rows in 'vendors' (113 rows):

vendorID | vendor_name | shop_name | lineID | badges | attendance | attendance_days | payment
-------------------------------------------------------------------------------------------------------
1 | None | อิงสร้อยผ้าโพก | None | ["เครื่องประดับ"] | 3 | None | None
2 | None | โอเสื้อยืด,เชิ้ต,แว่นตา | None | [] | 3 | None | None
3 | None | ไอซ์ stronger.store25 เสื้อยืดแจ็คเก็ต | None | ["เสื้อผ้า"] | 3 | None | None
4 | None | พี่ตั้มหอยทากเสื้อยืดของเล่น | None | ["เสื้อผ้า", "ของเล่น"] | 3 | None | None
5 | None | แม่ต่ายเสื้อยืด | None | ["เสื้อผ้า", "ของเล่น"] | 3 | None | None
6 | None | แม่ต่ายแขนยาว | None | ["เสื้อผ้า"] | 3 | None | None
7 | None | โอมเพี้ยงเสื้อยืด | None | ["เสื้อผ้า"] | 3 | None | None
8 | None | พี่เบนซ์กระเป๋า | None | ["กระเป๋า"] | 3 | None | None
9 | None | ฟ่าง y2k | None | ["เสื้อผ้า", "y2k"] | 3 | None | None
10 | None | อาย daffodil.3store เสื้อฮิปปี้ | None | ["เสื้อผ้า"] | 3 | None | None
11 | None | บับเบิ้ลเสื้อ

In [ ]:
def update_attendance_to_three():
    conn = get_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("UPDATE vendors SET attendance = 3")
        conn.commit()
        print("All rows updated: attendance set to 3.")
    except mysql.connector.Error as err:
        print(f"Error updating rows: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
update_attendance_to_three()


Error updating rows: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction


In [14]:
conn = get_connection()
cursor = conn.cursor()

cursor.execute("DESCRIBE layouts;")
rows = cursor.fetchall()  # <- fetch all results first
for row in rows:
    print(row)

cursor.close()
conn.close()

('id', b'int', 'NO', bytearray(b'PRI'), None, 'auto_increment')
('name', b'varchar(255)', 'NO', bytearray(b''), None, '')
('data', b'json', 'YES', bytearray(b''), None, '')
('status', b'varchar(50)', 'YES', bytearray(b''), b'inactive', '')


In [12]:
print("Test script started")

def create_app():
    print("Inside create_app")
    return "app"

app = create_app()
print("App created:", app)


Test script started
Inside create_app
App created: app


In [17]:
conn = get_connection()
cursor = conn.cursor()

cursor.execute("SELECT data FROM layouts WHERE id = %s", (11,))
row = cursor.fetchone()
if row:
    data_json = row[0]
    # 2. Parse JSON string to dict
    data = json.loads(data_json)

    # 3. Add 'status' key to every vendor (each sub-dict)
    for vendor_key, vendor_info in data.items():
        vendor_info['status'] = "active"  # or any value you want

    # 4. Convert dict back to JSON string
    new_json_str = json.dumps(data, ensure_ascii=False)

    # 5. Update the database
    cursor.execute("UPDATE layouts SET data = %s WHERE id = %s", (new_json_str, 11))
    conn.commit()
    print("Updated data with 'status' key added.")
else:
    print("No record with id=3 found.")

conn.close()

Updated data with 'status' key added.


In [41]:
conn = get_connection()
cursor = conn.cursor()
cursor.execute("ALTER TABLE layouts MODIFY COLUMN status VARCHAR(50) DEFAULT 'inactive' ")
conn.commit()

In [40]:
import requests

url = "http://127.0.0.1:8080/deactivate_layout/13"


response = requests.post(url)
print(response.status_code)
print(response.json())


200
{'message': "Layout id=13 status updated to 'inactive'."}


In [8]:
conn = get_connection()
cursor = conn.cursor()
alter_table_query = """
            ALTER TABLE vendors 
            ADD COLUMN payment INT NULL;
            """
            
cursor.execute(alter_table_query)
conn.commit()
cursor.close()
conn.close()